In [59]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [54]:
data = pd.read_csv("../data/video_data.csv").head()
data

,Video Title,Video Like Count,Video View Count,Video Comment Count
0,Coding for 1 Month Versus 1 Year #shorts #coding,220649,4601638,2432
1,what 10 years of coding looks like…,1252936,21172887,10404
2,What is Coding?,23012,1570351,800
3,Programming vs Coding - What&#39;s the differe...,55928,1942385,1777
4,How to ACTUALLY learn to code... 7 Roadmaps fo...,120079,1918790,2861


In [55]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    return text

data["Video Title"] = data["Video Title"].apply(preprocess_text)

In [56]:
np.array(data["Video Title"])

array(['coding for  month versus  year shorts coding',
       'what  years of coding looks like ', 'what is coding ',
       'programming vs coding what  s the difference ',
       'how to actually learn to code  roadmaps for '], dtype=object)

In [79]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["Video Title"].tolist())
print(X)

  (0, 14)	0.41097956810005726
  (0, 20)	0.41097956810005726
  (0, 17)	0.41097956810005726
  (0, 10)	0.41097956810005726
  (0, 4)	0.33157572780533867
  (0, 2)	0.46307743968749077
  (1, 8)	0.4580647974959538
  (1, 9)	0.4580647974959538
  (1, 11)	0.4580647974959538
  (1, 21)	0.4580647974959538
  (1, 19)	0.3067713334232789
  (1, 2)	0.25806571676545303
  (2, 6)	0.7525151949161979
  (2, 19)	0.5039681962632367
  (2, 2)	0.42395393449691715
  (3, 3)	0.4580647974959538
  (3, 15)	0.4580647974959538
  (3, 18)	0.4580647974959538
  (3, 12)	0.4580647974959538
  (3, 19)	0.3067713334232789
  (3, 2)	0.25806571676545303
  (4, 13)	0.3218961114619563
  (4, 1)	0.3218961114619563
  (4, 7)	0.3218961114619563
  (4, 0)	0.3218961114619563
  (4, 16)	0.6437922229239126
  (4, 5)	0.3218961114619563
  (4, 4)	0.2597037559047736


In [86]:
y = [data["Video Like Count"], data["Video View Count"], data["Video Comment Count"]]
type(X), type(y)

(scipy.sparse._csr.csr_matrix, list)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [5, 3]